#NIRC2 preprocessing streamlined

Streamlined version of the notebook NIRC2_Preprocessing.


##Import <a id='import'></a>

In [ ]:
%reset

In [ ]:
from NIRC2_Preprocessing import *
import numpy as np

from vip.fits import display_array_ds9, write_fits
from vip.calib.badpixremoval import bp_clump_removal
from vip.pca import pca
from vip.pca.pca_local import annular_pca

%matplotlib inline
%load_ext autoreload
%autoreload 2

---

## Paths

Define the different paths.

In [ ]:
path_flat = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/GJ504/flats/'
path_sky = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/GJ504/sky/n0010.fits'
path_sci = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/GJ504/sci/'

# Folder where results are saved (if not required, use None). 
path_output = '/Users/Olivier/Documents/ULg/VORTEX/Data/RAW/Keck/GJ504/processed/'

##Preprocessing

Do you want to save the mflat and preprocessed images ?

In [ ]:
save = True

Let's run !

In [ ]:
# File list
filelist_flat = listing(path_flat, selection=False, ext='fits')
filelist_sci = listing(path_sci, selection=False, ext='fits')

# Let's go
mflat = masterFlat(filelist_flat, save=save, path_output=path_output, verbose=True, filtering=5)
sci_preprocessed = applyFlat(filelist_sci, mflat, save=save, path_output=path_output, verbose=True)
sky_preprocessed = applyFlat([path_sky], mflat, save=save, path_output=path_output, verbose=True)

## Vortex position

Display the sci and sky preprocessed images to estimate the position of a dust and the vortex.

In [ ]:
display_array_ds9(sci_preprocessed,sky_preprocessed)

Define the estimates for the dust and vortex positions. In addition, we define both the dust and vortex model. 

In [ ]:
dust_center = (592,606)
dust_box_size = 41
dust_fun = gauss2d_sym
dust_fun_parameters = [5]

vortex_center = (520,730)
vortex_box_size = 28
vortex_fun = gauss2d_sym
vortex_fun_parameters = [5]

Let's run !

In [ ]:
dust_options = {'center':dust_center, 'size':vortex_box_size, 'fun':dust_fun, 'parameters':dust_fun_parameters}
vortex_options = {'center':vortex_center, 'size':vortex_box_size, 'fun':vortex_fun, 'parameters':vortex_fun_parameters}

center_all = vortex_center_from_dust_signature(sci_preprocessed, sky_preprocessed, dust_options, vortex_options, 
                                               verbose=True)

## Registration and cropping

Define the size of the final cube. If **None**, the optimal size is automatically calculated.

In [ ]:
size = None

Let's run !

In [ ]:
cube_reg = cube_registration(sci_preprocessed, center_all, cube_output_size=size, save=True, 
                             verbose=True, path_output=path_output)

display_array_ds9(sci_preprocessed, cube_reg)

##Bad pixel removal

Under construction ...

##ADI or RDI PCA 

Get the parallactic angles

In [ ]:
angs = get_parallactic_angles(listing(path_sci), save=True, path_output=path_output)

Get the reference cube for RDI

In [ ]:
path_ref_for_RDI = '/Users/Olivier/Documents/ULg/VORTEX/Data/Cube_PSF_PA/Keck/HR8799/20150609/cube_HD219196_20150609.fits'
cube_ref = open_fits(path_ref_for_RDI)

Let's run !

In [ ]:
image = pca(cube_reg, angs, cube_ref=None, ncomp=3, svd_mode='randsvd', full_output=False)
display_array_ds9(image)

If you want to perform **annular_pca( )** instead of normal **pca()**:

In [ ]:
image = annular_pca(cube_reg, angs, ncomp=5, fwhm=8, svd_mode='randsvd', full_output=False)
display_array_ds9(image)

Save

In [ ]:
filename = 'final_image.fits'
write_fits(path_output+filename, image)